# Calibration

Calibration script following https://github.com/Jin93/CBHM/blob/master/Calibration.R

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import sys
from os.path import exists

sys.path.append('..')
sys.path.append('.')

In [3]:
import numpy as np
import pymc as pm
import arviz as az
from scipy.stats import binom
from scipy.stats import norm

In [4]:
# Set simulation settings and parameters
K = 6 # number of indications
alpha = 0.1 # significance level for the test
num_sim = 10 # number of simulations per simulation setting
Ni = 24 # maximum of total sample size for each indication group
Ni1 = 14 # stage-one sample size for each indication group

# Initialize matrices to record number of patients and responders in each indication group
nik = np.zeros((2, K), dtype=int)
rik = np.zeros((2, K))
nik[0,:] = Ni1

# Set null and target response rates
q0 = 0.2 # standard of care (null) response rate
q1 = 0.4 # target response rate
Qf = 0.05 # probability cut-off for interim analysis

In [5]:
# Define PyMC3 model for beta-binomial hierarchical model
def beta_binomial_model(n, Y):
    assert len(n) == len(Y)
    K = len(n)
    with pm.Model() as model:
        α = pm.Gamma('alpha', alpha=2, beta=0.5)
        β = pm.Gamma('beta', alpha=2, beta=0.5)
        θ = pm.Beta('mu', alpha=α, beta=β, shape=K)
        y = pm.Binomial('y', n=n, p=θ, observed=Y, shape=K)        
        return model

In [6]:
p0 = np.full(K, q0) # true response rate: set to null rate for all indications (null scenario)
posterior_ind = np.zeros((num_sim, K)) # matrix to record posterior probabilities of success for each indication in each simulation

In [7]:
draws = 1000
tune = 1000
chains = 1
pbar = False

In [8]:
for sim in range(num_sim):
    
    # Stage 1:
    n = nik[0, :]
    rik[0,:] = binom.rvs(n=n, p=p0) # generate response data
    
    Y = rik[0,:]
    num_baskets = len(n)
    
    with beta_binomial_model(n=n, Y=Y) as model:        
        trace = pm.sample(draws, tune=tune, chains=chains, progressbar=pbar)
        
    ## Interim analysis:
    stacked = az.extract(trace)        
    basket_probs = stacked.mu.values        
    posterior = np.zeros(num_baskets)
    for k in range(num_baskets):
        midpoint = (q0 + q1) / 2
        posterior[k] = np.mean(basket_probs[k, :] > midpoint)

    ## Futility stop:    
    stage2_stop = np.where(posterior < Qf)[0]
    stage2_cont = np.where(posterior >= Qf)[0]
    nik[1, stage2_cont] = Ni - Ni1 # enroll new patients

    # Store posterior of success from interim analysis
    posterior_ind[sim, stage2_stop] = posterior[stage2_stop] 

    # Stage 2:
    if len(stage2_cont) > 0:

        # generate response data
        rik[1, stage2_cont] = binom.rvs(n=nik[1, stage2_cont], p=p0[stage2_cont])
        ni = np.sum(nik[:, stage2_cont], axis=0)
        Y = np.sum(rik[:, stage2_cont], axis=0)
        num_baskets = len(ni)

        with beta_binomial_model(n=ni, Y=Y) as model2:
            trace2 = pm.sample(draws, tune=tune, chains=chains, progressbar=pbar)

        # Final decision
        stacked = az.extract(trace2)        
        basket_probs = stacked.mu.values        
        posterior = np.zeros(num_baskets)
        for k in range(num_baskets):
            posterior[k] = np.mean(basket_probs[k, :] > q0)

        posterior_ind[sim, stage2_cont] = posterior
        
    print(sim)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


0


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


1


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


...

7


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


8


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [alpha, beta, mu]
Sampling 1 chain for 1_000 tune and 1_000 draw iterations (1_000 + 1_000 draws total) took 2 seconds.


9


In [11]:
posterior_ind, alpha

(array([[0.003, 0.028, 0.654, 0.619, 0.876, 0.021],
        [0.167, 0.662, 0.944, 0.783, 0.94 , 0.949],
        [0.971, 0.436, 0.453, 0.028, 0.976, 0.134],
        [0.271, 0.587, 0.772, 0.432, 0.928, 0.012],
        [0.046, 0.726, 0.444, 0.426, 0.726, 0.003],
        [0.859, 0.277, 0.02 , 0.604, 0.014, 0.02 ],
        [0.923, 0.04 , 0.747, 0.435, 0.431, 0.446],
        [0.774, 0.773, 0.885, 0.882, 0.65 , 0.653],
        [0.28 , 0.415, 0.94 , 0.921, 0.286, 0.841],
        [0.356, 0.365, 0.502, 0.214, 0.341, 0.371]]),
 0.1)

In [14]:
Q_independent = np.quantile(posterior_ind, 1 - alpha)
Q_independent

0.9292